In [ ]:
# https://stackoverflow.com/questions/19664865/migrating-from-pulp-to-scipy

import numpy as np
import scipy.optimize as opt

def get_optimization(constraints, loss_factor):
    all_n_indices = []
    all_aw_indices = []

    variable_n_indices = {}
    variable_aw_indices = {}

    index_to_bucket = {}

    variables_n = { i: [] for i in range(1, 10) }
    variables_aw = { i: [] for i in range(1, 10) }

    conditions = []
    bounds = []
    guess = []

    counter = 0

    orig_all_weights = 0
    orig_all_weights_nongutted = 0
    orig_all_counts = 0

    for constraint_name, constraint in constraints.items():
        if len(constraint) == 4:
            range_buckets, range_count, range_total_weight, gutted_indicator = constraint
            
            if gutted_indicator == 'r':
                orig_all_weights_nongutted = orig_all_weights_nongutted + range_total_weight
                range_total_weight = range_total_weight * (1 - loss_factor)
        else:
            range_buckets, range_count, range_total_weight = constraint
            orig_all_weights_nongutted = orig_all_weights_nongutted + range_total_weight

        orig_all_weights = orig_all_weights + range_total_weight 
        orig_all_counts = orig_all_counts + range_count

        range_count = range_count / 10000
        range_total_weight = range_total_weight / 10000

        variable_n_indices[constraint_name] = []
        variable_aw_indices[constraint_name] = []

        for bucket in range(range_buckets[0], range_buckets[1]):
            if range_buckets[1] - range_buckets[0] > 1:
                guess_n = range_count * (bucket + 0.5) / (np.sum(range(range_buckets[0], range_buckets[1])) + 0.5 * (range_buckets[1] - range_buckets[0]))
                guess_aw = bucket + 0.5
            else:
                guess_n = range_count
                guess_aw = range_total_weight / range_count

            variable_n_indices[constraint_name].append(counter)
            variables_n[bucket].append(counter)
            all_n_indices.append(counter)
            guess.append(guess_n)
            bounds.append((0, range_count))
            index_to_bucket[counter] = bucket

            counter = counter + 1

            variable_aw_indices[constraint_name].append(counter)
            variables_aw[bucket].append(counter)
            all_aw_indices.append(counter)
            guess.append(guess_aw)
            bounds.append((bucket, bucket + 1))
            index_to_bucket[counter] = bucket

            counter = counter + 1

        variable_n_indices[constraint_name] = np.array(variable_n_indices[constraint_name])
        variable_aw_indices[constraint_name] = np.array(variable_aw_indices[constraint_name])

        n_indices = variable_n_indices[constraint_name].astype(int)
        aw_indices = variable_aw_indices[constraint_name].astype(int)

    #     print(n_indices)
    #     print(w_indices)

        conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[n_indices]) - range_count) })
        conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[aw_indices] * x[n_indices]) - range_total_weight) })

    #     for index, bucket in enumerate(range(range_buckets[0], range_buckets[1])):
    #         n_index = variable_n_indices[constraint_name][index]
    #         aw_index = variable_aw_indices[constraint_name][index]
    # #         print(bucket, n_index, w_index)

    #         x1 = min((bucket + 1) - x[w_index] / x[n_index], min(0, x[w_index] / x[n_index] - bucket))
    #         print(x1)

    # #         conditions.append({ 'type': 'ineq', 'fun': lambda x: min(0.000001, x[w_index] / x[n_index] - bucket)  })
    # #         conditions.append({ 'type': 'ineq', 'fun': lambda x: min(0.000001, (bucket + 1) - x[w_index] / x[n_index]) })
    #         conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 + x1 })

    for bucket in variables_n:
        variables_n[bucket] = np.array(variables_n[bucket])
        variables_aw[bucket] = np.array(variables_aw[bucket])

        n_indices = variables_n[bucket].astype(int)
        aw_indices = variables_aw[bucket].astype(int)

        if len(n_indices) == 0:
            continue

    #     print(bucket)
    #     print('a', n_indices, np.sum(n_indices))
    #     print('b', w_indices, np.sum(w_indices))

        conditions.append({ 'type': 'ineq', 'fun': lambda x: np.sum(x[aw_indices] * x[n_indices]) - bucket * np.sum(x[n_indices]) })
        conditions.append({ 'type': 'ineq', 'fun': lambda x: (bucket + 1) * np.sum(x[n_indices]) - np.sum(x[aw_indices] * x[n_indices]) })

    conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[all_n_indices]) - orig_all_counts / 10000) })
    conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[all_aw_indices] * x[all_n_indices]) - orig_all_weights / 10000) })

    def objective_function(x):
        objective = 0

    #     for bucket in variables_n:
    #         n_indices = variables_n[bucket].astype(int)
    #         aw_indices = variables_aw[bucket].astype(int)

    #         for index, n_index in enumerate(variables_n[bucket]):
    #             aw_index = aw_indices[index]

    #             x1 = bucket - (x[w_index] / x[n_index])
    #             x2 = (x[w_index] / x[n_index]) - (bucket + 1)

    #             print(x1, x2)
    #             print(x1, x2, 1 - .5 * (1 + x1 + np.sqrt((x1 - 1) ^ 2 + .01) ) + 1 - .5 * (1 + x2 + np.sqrt((x2 - 1) ^ 2 + .01) ))

    #             objective = objective + .1 * (1 - .5 * (1 + x1 + np.sqrt((x1 - 1) ** 2 + .01) ) + 1 - .5 * (1 + x2 + np.sqrt((x2 - 1) ** 2 + .01) ))
    #             objective = objective + .01 * (-np.log(1 + np.exp(-(-x1))) + -np.log(1 + np.exp(-(-x2))))
    #             a = (-np.log(1 + np.exp(-(x[w_index] / x[n_index] - bucket))) + -np.log(1 + np.exp(-((bucket + 1) - x[w_index] / x[n_index]))))
    #             if not np.isfinite(a):
    #                 print(x[w_index], x[n_index], bucket, x[w_index] / x[n_index] - bucket)
    #             print('a', n_indices, n_index, np.sum(x[n_indices]), x[n_index])
    #             print('b', w_indices, w_index, np.sum(x[w_indices]), x[w_index])
    #             print('c',(x[w_index] / x[n_index] - np.sum(x[w_indices]) / np.sum(x[n_indices])))

                # TODO: Remove this as I think it's not correct
    #             objective = objective + np.abs((x[w_index] / x[n_index]) - (np.sum(x[w_indices]) / np.sum(x[n_indices])))
    #             objective = objective + x[n_index] * np.abs((x[w_index] / x[n_index]) - (np.sum(x[w_indices]) / np.sum(x[n_indices])))
    #             objective = objective + np.abs((x[n_index] / x[n_index]) - (np.sum(x[n_indices]) / all_counts))

        for constraint_name, constraint in constraints.items():
            if len(constraint) == 4:
                range_buckets, range_count, range_total_weight, gutted_indicator = constraint
                
                if gutted_indicator == 'r':
                    range_total_weight = range_total_weight * (1 - loss_factor)
            else:
                range_buckets, range_count, range_total_weight = constraint

            range_count = range_count / 10000
            range_total_weight = range_total_weight / 10000

            n_indices = variable_n_indices[constraint_name].astype(int)
            aw_indices = variable_aw_indices[constraint_name].astype(int)

            total_subset_count = 0

            for n_index in n_indices:
                bucket = index_to_bucket[n_index]
                bucket_indices = variables_n[bucket]
                total_subset_count = total_subset_count + np.sum(x[bucket_indices])

            for index, n_index in enumerate(n_indices):
                aw_index = aw_indices[index]

                bucket = index_to_bucket[n_index]
                bucket_indices = variables_n[bucket]

                objective = objective + np.abs((x[n_index] / np.sum(x[n_indices]) - np.sum(x[bucket_indices]) / total_subset_count))
                #objective = objective + x[n_index] * np.abs((x[n_index] / np.sum(n_indices) - np.sum(x[bucket_indices]) / total_subset_count))

            objective = objective + np.abs(np.sum(x[n_indices]) - range_count) / range_count
            objective = objective + np.abs(np.sum(x[aw_indices] * x[n_indices]) - range_total_weight) / range_total_weight

        return objective

    # print(guess)
    # print(bounds)
    # print(conditions)
#     print(all_weights, all_co#unts)

    results = opt.minimize(objective_function, guess, method='SLSQP', bounds=bounds, constraints = conditions, options={'disp': True})
#     print(results)

    def get_results(results):
        x = results.x

        all_weight = 0
        all_count = 0

        for bucket in variables_n:
            n_indices = variables_n[bucket].astype(int)
            aw_indices = variables_aw[bucket].astype(int)

            total_weight = np.sum(x[aw_indices] * x[n_indices]) * 10000
            total_count = np.sum(x[n_indices]) * 10000

            all_weight = all_weight + total_weight
            all_count = all_count + total_count

        print()
        
        print('Total Fish: %i' % (orig_all_counts, ))
        print('Total Non-Gutted Weight: %0.2f' % (orig_all_weights_nongutted, ))
        print('Total Gutted Weight: %0.2f' % (orig_all_weights, ))
        print('Average Gutted Weight: %0.2f' % (all_weight / all_count * 1000, ))
            
        print()
            
        print('%0.2f vs %0.2f, %0.2f vs %0.2f' % (all_weight, orig_all_weights, all_count, orig_all_counts))

        print()

        for bucket in variables_n:
            n_indices = variables_n[bucket].astype(int)
            aw_indices = variables_aw[bucket].astype(int)

            total_weight = np.sum(x[aw_indices] * x[n_indices]) * 10000
            total_count = np.sum(x[n_indices]) * 10000

            print('%i - %i: %0.2f, %0.2f, %0.2f' % (bucket * 1000, (bucket + 1) * 1000, total_weight / total_count * 1000, total_count / all_count * 100, total_weight / all_weight * 100))

        print()

        for constraint_name in variable_n_indices:
            n_indices = variable_n_indices[constraint_name].astype(int)
            aw_indices = variable_aw_indices[constraint_name].astype(int)

            total_weight = np.sum(x[aw_indices] * x[n_indices]) * 10000
            total_count = np.sum(x[n_indices]) * 10000

            constraint = constraints[constraint_name]
            
            if len(constraint) == 4:
                range_buckets, range_count, range_total_weight, gutted_indicator = constraint
                
                if gutted_indicator == 'r':
                    range_total_weight = range_total_weight * (1 - loss_factor)
            else:
                range_buckets, range_count, range_total_weight = constraint

            abs_weight_diff = np.abs((range_total_weight - total_weight) / range_total_weight) * 100
            abs_count_diff = np.abs((range_count - total_count) / range_count) * 100

            print(abs_weight_diff < 5, abs_count_diff < 5, constraint, '%0.2f vs %0.2f (%0.2f)' % (range_total_weight, total_weight, abs_weight_diff), '%0.2f vs %0.2f (%0.2f)' % (range_count, total_count, abs_count_diff))

        print()

        for constraint_name in variable_n_indices:
            constraint = constraints[constraint_name]
            
            if len(constraint) == 4:
                range_buckets, range_count, range_total_weight, gutted_indicator = constraint
                
                if gutted_indicator == 'r':
                    range_total_weight = range_total_weight * (1 - loss_factor)
            else:
                range_buckets, range_count, range_total_weight = constraint

            n_indices = variable_n_indices[constraint_name].astype(int)
            aw_indices = variable_aw_indices[constraint_name].astype(int)

            subset_bucket_count = 0
            for index, n_index in enumerate(n_indices):
                bucket = index_to_bucket[n_index]
                bucket_n_indices = variables_n[bucket].astype(int)
                bucket_count = np.sum(x[bucket_n_indices]) * 10000
                subset_bucket_count = subset_bucket_count + bucket_count

            for index, n_index in enumerate(n_indices):
                aw_index = aw_indices[index]

                bucket = index_to_bucket[n_index]

                bucket_n_indices = variables_n[bucket].astype(int)
                bucket_count = np.sum(x[bucket_n_indices]) * 10000

                weight = x[aw_index]
                count = x[n_index] * 10000
                avg_weight = weight
                valid = (avg_weight > bucket) and (avg_weight < (bucket + 1))

                print(valid, constraint_name, bucket, '%0.3f @ %0.3f (%0.3f vs %0.2f)' % (avg_weight, count, count / range_count * 100, bucket_count / subset_bucket_count * 100))

    return get_results(results)

In [ ]:
# aplavika = {
#     '0-10': ((2, 5), 213, 826.00, 'r'),
#     '2-3': ((2, 3), 159, 437.80),
#     '3-10': ((3, 5), 667, 1667.2),
#     '3-4': ((3, 4), 4089, 14856.00),
#     '3-6': ((3, 6), 4279, 19030.50),
#     '4.5-5.0': ((4, 5), 540, 2553.80),
#     '4-5': ((4, 5), 14872, 67174.40),
#     '5-6': ((5, 6), 14264, 77238.80),
#     '6+': ((6, 8), 618, 4071.80),
#     '6-7': ((6, 7), 4640, 29502.20),
#     '7-8': ((7, 8), 730, 5323.00),
#     '8-9': ((8, 9), 49, 402.20),
#     '9+': ((9, 10), 2, 18.30),
# }

# get_optimization(aplavika, 0.14)

# tittelsnes = {
#     's1-2': ((1, 2), 1331, 2327.24),
#     's2.5-3': ((2, 3), 2096, 5843.52),
#     's2-3': ((2, 3), 11421, 29690.88),
#     's3-4': ((3, 4), 23674, 83086.56),
#     's3-4..': ((3, 4), 2592, 9726.43),
#     's4.7-5': ((4, 5), 709, 3432.99),
#     's4.4-4.8': ((4, 5), 319, 1447.16),
#     's4.5-5': ((4, 5), 1420, 6775.35),
#     's4-5': ((4, 5), 20009, 89067.62),
#     's5-5.2': ((5, 6), 73, 372.47),
#     's5-6': ((5, 6), 11784, 64285.36),
#     's6-7': ((6, 7), 3873, 24939.11),
#     's7-8': ((7, 8), 860, 6378.96),
#     's8-9': ((8, 9), 104, 875.44),
#     's9+': ((9, 10), 10, 95.39),
    
#     'o1-2': ((1, 2), 1511, 2574.29),
#     'o2-3': ((2, 3), 6327, 16271.07),
#     'o3-4': ((3, 4), 6210, 21607.36),
#     'o4-5': ((4, 5), 3109, 13867.64),
#     'o5-6': ((5, 6), 1110, 6028.70),
#     'o6-7': ((6, 7), 279, 1795.51),
#     'o7-8': ((7, 8), 21, 153.79),
#     'o8-9': ((8, 9), 6, 53.7),
    
#     'pa1.8-3': ((1, 3), 2064, 5003.63),
#     'pa1-1.8': ((1, 2), 425, 632.62),
#     'pa3-5': ((3, 5), 1752, 6618.08),
#     'pa5+': ((5, 8), 284, 1661.91),
    
#     'pb1-3': ((1, 3), 185, 400.07),
#     'pb3+': ((3, 6), 181, 749.85),
    
#     'u0-20': ((1, 3), 546, 833.46, 'r'),
    
#     'd0-20': ((2, 5), 855, 3306.42, 'r')
# }

# get_optimization(tittelsnes, 0.16)

# langoy = {
#     's1-2': ((1, 2), 287, 497.90),
#     's2-3': ((2, 3), 2813, 7494.69),
#     's3-4': ((3, 4), 8336, 29522.20),
#     's4-5': ((4, 5), 8319, 36998.00),
#     's5-6': ((5, 6), 5180, 28236.53),
#     's6-7': ((6, 7), 3263, 20945.60),
#     's7-8': ((7, 8), 1459, 10787.60),
#     's8-9': ((8, 9), 440, 3670.40),
#     's9+': ((9, 10), 106, 1001.80),
    
#     'o2-3': ((2, 3), 400, 1059.60),
#     'o3-4': ((3, 4), 994, 3511.40),
#     'o4-5': ((4, 5), 900, 4017.70),
#     'o5-6': ((5, 6), 551, 3004.20),
#     'o6-7': ((6, 7), 297, 1911.60),
#     'o7-8': ((7, 8), 147, 1084.50),
#     'o8-9': ((8, 9), 51, 427.10),
#     'o9+': ((9, 10), 10, 94.00),
    
#     'p1-2': ((1, 2), 72, 122.50),
#     'p2-3': ((2, 3), 345, 903.20),
#     'p3-4': ((3, 4), 987, 3465.70),
#     'p4-5': ((4, 5), 983, 4376.20),
#     'p5-6': ((5, 6), 731, 3995.20),
#     'p6+': ((6, 9), 1003, 7024.80),
    
#     'u': ((1, 5), 301, 915.30)
# }

# get_optimization(langoy, 0.16)

# eldviktaren1 = {
#     'g': ((2, 6), 33, 135.00, 'r'),
    
#     's1-2': ((1, 2), 109, 190.80),
#     's2-3': ((2, 3), 4431, 12135.10),
#     's3-4': ((3, 4), 14223, 49290.80),
#     's4.0-4.5': ((4, 5), 12, 49.70),
#     's4.5-5.0': ((4, 5), 2, 9.40),
#     's4-5': ((4, 5), 3397, 14534.60),
#     's5-6': ((5, 6), 87, 446.00),
#     's6-7': ((6, 7), 6, 39.90),
    
#     'v': ((2, 6), 47, 187, 'r'),
    
#     'pa0-3': ((1, 3), 297, 659.80),
#     'pa3-4': ((3, 4), 202, 695.90),
#     'pa4-5': ((4, 5), 60, 258.10),
    
#     'pb0-3': ((1, 3), 301, 833.74),
#     'pb3-4': ((3, 4), 1675, 5763.40),
#     'pb4-5': ((4, 5), 176, 725.10),
    
#     'f': ((2, 5), 78, 256.90)
# }

# get_optimization(eldviktaren1, 0.16)

# eldviktaren2 = {
#     'g': ((2, 6), 48, 177.00, 'r'),
    
#     's1-2': ((1, 2), 197, 349.40),
#     's2.5-3.0': ((2, 3), 1403, 3931.20),
#     's2-3': ((2, 3), 8219, 22361.50),
#     's3.5-4.0': ((3, 4), 483, 1772.80),
#     's3-4': ((3, 4), 26954, 92478.90),
#     's4.5-5.0': ((4, 5), 325, 1508.10),
#     's4-5': ((4, 5), 4720, 20003.50),
#     's5-6': ((5, 6), 97, 499.60),
#     's6-7': ((6, 7), 12, 79.00),
    
#     'v': ((2, 6), 53, 210.00, 'r'),
    
#     'pa0-3': ((1, 3), 475, 1091.70),
#     'pa3-4': ((3, 4), 328, 1128.00),
#     'pa4-5': ((4, 5), 112, 480.00),
#     'pa5-6': ((5, 6), 2, 10.10),
#     'pa6+': ((2, 5), 6, 18.30),
    
#     'pb0-3': ((1, 3), 689, 1896.30),
#     'pb3-4': ((3, 4), 2783, 9556.80),
#     'pb4-5': ((4, 5), 309, 1293.70),
#     'pb5-6': ((5, 6), 9, 42.70),
    
#     'f': ((2, 5), 198, 638.20)
# }

# get_optimization(eldviktaren2, 0.16)

# eldviktaren3 = {
#     'g': ((2, 6), 109, 431.00, 'r'),
    
#     's1-2': ((1, 2), 157, 276.40),
#     's2-3': ((2, 3), 9358, 25722.00),
#     's3.5-4.0': ((3, 4), 161, 592.90),
#     's3-4': ((3, 4), 27572, 94638.91),
#     's4.0-4.5': ((4, 5), 220, 918.80),
#     's4.5-5.0': ((4, 5), 134, 619.60),
#     's4-5': ((4, 5), 4571, 19482.30),
#     's5-6': ((5, 6), 83, 429.30),
    
#     'v': ((2, 6), 183, 727.00, 'r'),
    
#     'pa0-3': ((1, 3), 356, 863.40),
#     'pa3-4': ((3, 4), 362, 1235.20),
#     'pa4-5': ((4, 5), 91, 381.30),
    
#     'pb0-3': ((1, 3), 802, 2197.20),
#     'pb3-4': ((3, 4), 2679, 9190.40),
#     'pb4-5': ((4, 5), 293, 1226.80),
#     'pb5-6': ((5, 6), 4, 21.00),
    
#     'f': ((2, 5), 179, 580.10)
# }

# get_optimization(eldviktaren3, 0.16)

# eldviktaren4 = {
#     'g': ((2, 6), 118, 461.00, 'r'),
    
#     's1-2': ((1, 2), 177, 320.40),
#     's2.5-3.0': ((2, 3), 1198, 3340.40),
#     's2-3': ((2, 3), 9131, 24973.00),
#     's3-4': ((3, 4), 26434, 90420.46),
#     's4-5': ((4, 5), 4105, 17493.00),
#     's5-6': ((5, 6), 67, 348.50),
    
#     'v': ((2, 6), 297, 1160.00, 'r'),
    
#     'pa0-3': ((1, 3), 415, 983.50),
#     'pa3-4': ((3, 4), 345, 1167.30),
#     'pa4-5': ((4, 5), 62, 266.00),
#     'pa5-6': ((5, 6), 4, 21.90),
    
#     'pb0-3': ((1, 3), 816, 2243.90),
#     'pb3-4': ((3, 4), 2776, 9554.20),
#     'pb4-5': ((4, 5), 251, 1049.30),
    
#     'f': ((2, 5), 232, 738.00)
# }

# # get_optimization(eldviktaren4, 0.16)

# # TODO: Smarter combination of multiple days taking starvation into account
# eldviktaren = {}

# eldviktaren_sources = [eldviktaren1, eldviktaren2, eldviktaren3, eldviktaren4]

# for index, source in enumerate(eldviktaren_sources):
#     for key, value in source.items():
#         new_key = '%i-%s' % (index, key)
#         eldviktaren[new_key] = value
        
# get_optimization(eldviktaren, 0.16)

# movikodden = {
#     's2-3': ((2, 3), 1962, 5044.45),
#     's3-4': ((3, 4), 3878, 13763.43),
#     's4-5': ((4, 5), 8301, 37246.97),
#     's5-6': ((5, 6), 4415, 23566.48),
#     's6-7': ((6, 7), 627, 3913.80),
#     's7-8': ((7, 8), 21, 151.28),
    
#     'r1-2': ((1, 2), 264, 475.41),
#     'r1-3': ((1, 3), 212, 526.39),
#     'r3-5': ((3, 5), 189, 764.24),
#     'r5-8': ((5, 8), 34, 185.66),
    
#     'rl1-3': ((1, 3), 15, 36.99),
#     'rl3-6': ((3, 6), 104, 425.15),
    
#     'w': ((3, 6), 20, 86),
    
#     'd': ((3, 6), 15, 65),
    
#     'f': ((3, 6), 4, 17)
# }

# get_optimization(movikodden, 0.18)

varholmenp5 = {
#     'pa0-1': ((1, 2), 3, 2.67),
    'pa1-2': ((1, 2), 235, 392.62),
    'pa2-3': ((2, 3), 1137, 2948.06),
    'pa3-4': ((3, 4), 3918, 14002.98),
    'pa4-5': ((4, 5), 5307, 23711.90),
    'pa5-6': ((5, 6), 2610, 14037.44),
    'pa6-7': ((6, 7), 484, 3056.34),
    'pa7-8': ((7, 8), 52, 377.04),
    'pa8-9': ((8, 9), 5, 40.86),
    
    'pb1-2.5': ((1, 3), 731, 1934.82),
    'pb3-4': ((3, 4), 3618, 12898.66),
    'pb4-5': ((4, 5), 4407, 19631.16),
    'pb5-6': ((5, 6), 1800, 9640.46),
    'pb6-7': ((6, 7), 321, 2020.42),
    'pb7-8': ((7, 8), 24, 174.32),
    'pb8-9': ((8, 9), 1, 8.04),
    
    'u1-2.5': ((1, 3), 7, 10.85),
    'u2.5+': ((3, 6), 56, 263.54),
    
    'sk2-3': ((2, 3), 64, 172.54),
    'sk4-5': ((4, 5), 5, 22.70),
    
    's1-2': ((1, 2), 97, 159.44),
    's2-3': ((2, 3), 1346, 3608.24),
    's3-4': ((3, 4), 13658, 50436.21),
    's4-5': ((4, 5), 40538, 185173.43),
    's4.5-5': ((4, 5), 810, 3847.22),
    's5-6': ((5, 6), 42292, 230652.74),
    's6-7': ((6, 7), 15746, 100464.18),
    's7-8': ((7, 8), 2769, 20292.78),
    's8-9': ((8, 9), 227, 1882.10),
    's9+': ((9, 10), 8, 73.38)
}

get_optimization(varholmenp5, 0.16)